## Dependencies

In [1]:
import numpy as np
import pandas as pd

## Import and Prep Data

In [8]:
# Player data from CSV
stats = pd.read_csv("../resources/game_skater_stats.csv")
stats.columns

Index(['game_id', 'player_id', 'primaryPosition', 'team_id', 'timeOnIce',
       'assists', 'goals', 'shots', 'hits', 'powerPlayGoals',
       'powerPlayAssists', 'penaltyMinutes', 'faceOffWins', 'faceoffTaken',
       'takeaways', 'giveaways', 'shortHandedGoals', 'shortHandedAssists',
       'blocked', 'plusMinus', 'evenTimeOnIce', 'shortHandedTimeOnIce',
       'powerPlayTimeOnIce'],
      dtype='object')

In [9]:
# Group player ids to get total player stats
player_stats_avg = stats.groupby(["player_id","primaryPosition"],as_index=False).mean().fillna(0)
player_stats_sum = stats.groupby("player_id").sum()

In [10]:
X_df = player_stats_avg.drop(columns=["game_id","team_id","player_id","primaryPosition"])

In [15]:
# Establish X, y
X = X_df.values
y = player_stats_avg["primaryPosition"]

## Create Training and Testing sets

In [23]:
# Import Dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [24]:
# Convert Player Positions (C, W, D) to numerical values
label_encoder = LabelEncoder()
label_encoder.fit(y)
encoded_y = label_encoder.transform(y)

In [53]:
## CLASSES
# 0 - C (Center)
# 1 - D (Defense)
# 2 - W (Winger)

In [39]:
# Create train and test splits
X_train, X_test, y_uncat_train, y_uncat_test = train_test_split(X, encoded_y, random_state=42)

In [40]:
# Categorize the y
y_train = to_categorical(y_uncat_train)
y_test = to_categorical(y_uncat_test)

## Build a Deep Neural Network

In [64]:
# Import Dependecies
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dense

In [65]:
# Create an empty sequential model
model = Sequential()

In [66]:
# Add the first layer where the input dimensions are the columns of the training data
model.add(Dense(100, activation="relu", input_dim=X_train.shape[1]))

In [67]:
# Add a second hidden layer
model.add(Dense(100, activation="relu"))

In [68]:
# Add output layer
model.add(Dense(units=y_train.shape[1], activation="softmax"))

In [69]:
# Compile the model using categorical_crossentropy for the loss function, the adam optimizer,
# and add accuracy to the training metrics
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

In [70]:
# Use the training data to fit (train) the model
model.fit(X_train, y_train, epochs=100, shuffle=True, verbose=2)

Epoch 1/100
79/79 - 0s - loss: 18.5132 - accuracy: 0.3811
Epoch 2/100
79/79 - 0s - loss: 1.8678 - accuracy: 0.4272
Epoch 3/100
79/79 - 0s - loss: 1.6156 - accuracy: 0.4869
Epoch 4/100
79/79 - 0s - loss: 1.2103 - accuracy: 0.5923
Epoch 5/100
79/79 - 0s - loss: 1.0687 - accuracy: 0.6090
Epoch 6/100
79/79 - 0s - loss: 0.9091 - accuracy: 0.6575
Epoch 7/100
79/79 - 0s - loss: 1.0300 - accuracy: 0.6348
Epoch 8/100
79/79 - 0s - loss: 1.0787 - accuracy: 0.6201
Epoch 9/100
79/79 - 0s - loss: 1.0134 - accuracy: 0.6480
Epoch 10/100
79/79 - 0s - loss: 0.9875 - accuracy: 0.6404
Epoch 11/100
79/79 - 0s - loss: 1.1852 - accuracy: 0.6639
Epoch 12/100
79/79 - 0s - loss: 1.6921 - accuracy: 0.5732
Epoch 13/100
79/79 - 0s - loss: 0.9973 - accuracy: 0.6858
Epoch 14/100
79/79 - 0s - loss: 0.8491 - accuracy: 0.6862
Epoch 15/100
79/79 - 0s - loss: 1.4278 - accuracy: 0.6277
Epoch 16/100
79/79 - 0s - loss: 1.1021 - accuracy: 0.6555
Epoch 17/100
79/79 - 0s - loss: 1.3908 - accuracy: 0.6595
Epoch 18/100
79/79 - 0

## Evaluate the Model

In [71]:
# Evaluate the model using the training data
model_loss, model_accuracy = model.evaluate(X_test, y_test, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

27/27 - 0s - loss: 0.3864 - accuracy: 0.8582
Loss: 0.38638192415237427, Accuracy: 0.8581644892692566
